In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
273,AFG,Asia,Afghanistan,2020-09-29,39239.0,12.0,23.571,1456.0,3.0,1.714,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
478,ALB,Europe,Albania,2020-09-29,13391.0,132.0,122.286,380.0,3.0,2.286,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
752,DZA,Africa,Algeria,2020-09-29,51213.0,146.0,170.000,1719.0,5.0,5.714,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-28,France,552048.0,0.0,31775.0,0.0,0.0,0.0
0,2020-09-28,China,90505.0,0.0,4744.0,0.0,0.0,0.0
0,2020-09-28,Italy,311486.0,0.0,35867.0,0.0,0.0,0.0
0,2020-09-28,Spain,721974.0,0.0,31240.0,0.0,0.0,0.0
0,2020-09-28,United States,7194127.0,0.0,205990.0,0.0,0.0,0.0
0,2020-09-28,World,33726644.0,0.0,1008533.0,0.0,0.0,0.0
0,2020-09-28,United Kingdom,439582.0,0.0,42046.0,0.0,0.0,0.0
0,2020-09-28,Germany,286658.0,0.0,9469.0,0.0,0.0,0.0
0,2020-09-28,Iran,449641.0,0.0,25800.0,0.0,0.0,0.0
0,2020-09-28,Turkey,315651.0,0.0,8055.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-29,France,552048.0,542639.0,31775.0,31808.0,9409.0,-33.0
0,2020-09-29,China,90505.0,90505.0,4744.0,4739.0,0.0,5.0
0,2020-09-29,Italy,311486.0,311364.0,35867.0,35851.0,122.0,16.0
0,2020-09-28,Spain,721974.0,748266.0,31240.0,31411.0,-26292.0,-171.0
0,2020-09-29,United States,7194127.0,7148044.0,205990.0,205070.0,46083.0,920.0
0,2020-09-29,World,33726644.0,33423469.0,1008533.0,1002678.0,303175.0,5855.0
0,2020-09-29,United Kingdom,439582.0,439013.0,42046.0,42001.0,569.0,45.0
0,2020-09-29,Germany,286658.0,287421.0,9469.0,9471.0,-763.0,-2.0
0,2020-09-29,Iran,449641.0,449960.0,25800.0,25779.0,-319.0,21.0
0,2020-09-29,Turkey,315651.0,315845.0,8055.0,8062.0,-194.0,-7.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")